# 2. Modeling

### 2.0 Load Train, Val, Test (Numpy Array)

- Gray Scale : Data_gray 폴더 안에 .npy 파일이 있으면 된다.
- RGB Scale : Data 폴더 안에 .npy 파일이 있으면 된다.

#### Scale 설정
- scale = 'gray'
- scale = 'rgb'

In [1]:
scale = 'rgb'
data_dir = 'Data_class_integrate_4'
channel = 3

In [2]:
!ls {data_dir}

X_test.npy  X_train.npy  X_val.npy  y_test.npy	y_train.npy  y_val.npy


#### Load Data

In [3]:
import os
import numpy as np

if os.path.exists(data_dir):
    X_train = np.load(data_dir + '/X_train.npy')
    y_train = np.load(data_dir + '/y_train.npy')
    X_val = np.load(data_dir + '/X_val.npy')
    y_val = np.load(data_dir + '/y_val.npy')
    X_test = np.load(data_dir + '/X_test.npy')
    y_test = np.load(data_dir + '/y_test.npy')

    print(X_train.shape, y_train.shape)
    print(X_val.shape, y_val.shape)
    print(X_test.shape, y_test.shape)

(300, 64, 128, 128, 3) (300, 4)
(100, 64, 128, 128, 3) (100, 4)
(100, 64, 128, 128, 3) (100, 4)


### 정규화

In [4]:
X_train = X_train.astype(float) / 255
X_val = X_val.astype(float) / 255
X_test = X_test.astype(float) / 255

### 2.1 VGG16 + LSTM

#### 모델 이름 설정

In [5]:
model_name = 'vgg16_lstm_class_integrate_new'

#### Parameters

In [6]:
epochs = 10
batch_size = 2

### 2.1.1 Model Define

In [7]:
IMG_FRAMES = 64
IMG_WIDTH = 128
IMG_HEIGHT = 128
CHANNEL_RGB = 3

CLASSES = 4

In [8]:
from keras.applications.vgg16 import VGG16
from keras import Sequential
from keras.layers import TimeDistributed, LSTM, Dense, Dropout, Flatten, GlobalAveragePooling2D

vgg = VGG16(include_top=False, weights='imagenet', input_shape= (IMG_HEIGHT, IMG_WIDTH, CHANNEL_RGB))

for layer in vgg.layers[:-4]: # Fine tuning
    layer.trainable = False
    
model = Sequential()

model.add(TimeDistributed(vgg, input_shape = (IMG_FRAMES, IMG_HEIGHT, IMG_WIDTH, CHANNEL_RGB)))

model.add(TimeDistributed(GlobalAveragePooling2D()))

model.add(LSTM(256, activation='relu', return_sequences=False))

model.add(Dense(1024, activation='relu'))
model.add(Dense(CLASSES, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 64, 4, 4, 512)     14714688  
_________________________________________________________________
time_distributed_1 (TimeDist (None, 64, 512)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               787456    
_________________________________________________________________
dense (Dense)                (None, 1024)              263168    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 4100      
Total params: 15,769,412
Trainable params: 8,134,148
Non-trainable params: 7,635,264
_________________________________________________________________


### 2.1.2 Model Compile

In [9]:
from keras import optimizers

model.compile(loss = 'categorical_crossentropy',
              optimizer = optimizers.Adam(lr=0.0001), # defalut : 0.001
              metrics = ['accuracy'])

### 2.1.3 Model Fit

In [10]:
%%time

History = model.fit(X_train, y_train,
                   epochs = epochs,
                   batch_size = batch_size,
                   validation_data = (X_val, y_val))

Epoch 1/10
 10/150 [=>............................] - ETA: 16:12 - loss: 1.4145 - accuracy: 0.1829

KeyboardInterrupt: 

### 2.1.4 Fitted Model Visualization

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(History.history['loss']) + 1)

plt.figure(figsize = (9, 6))
plt.plot(epochs, History.history['loss'])
plt.plot(epochs, History.history['val_loss'])

plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Validation Loss'])
plt.grid()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(History.history['loss']) + 1)

plt.figure(figsize = (9, 6))
plt.plot(epochs, History.history['accuracy'])
plt.plot(epochs, History.history['val_accuracy'])

plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.grid()
plt.show()

### 2.1.5 Model Evaluate

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test,
                                batch_size = batch_size)

print('Loss = {:.5f}'.format(loss))
print('Accuracy = {:.5f}'.format(accuracy))

### 2.1.6 Model Save

In [ ]:
save_dir = 'Models'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

final_model_name = '/' + scale + '_' + model_name + '.h5'

model.save(save_dir + final_model_name)

### 2.1.7 Saved Model Test

In [ ]:
from keras.models import load_model

rgb_cnn_lstm = load_model(save_dir + final_model_name)

loss, accuracy = rgb_cnn_lstm.evaluate(X_test, y_test,
                                batch_size = batch_size)

print('Loss = {:.5f}'.format(loss))
print('Accuracy = {:.5f}'.format(accuracy))